# LDA2Vec
Adapted from tensorflow implementation: 

(New) https://github.com/nateraw/Lda2vec-Tensorflow

(Old) https://github.com/meereeum/lda2vec-tf

Import Libraries (Must restart after installing pyLDAvis)

In [ ]:
!pip install pyLDAvis

     |████████████████████████████████| 1.7 MB 5.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 15.7 MB 199 kB/s 
     |████████████████████████████████| 11.3 MB 36.8 MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136897 sha256=62d7faf3364e53426c2a1852f0609e845b5cc6d8ace085346f387edc6c8592ef
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take int

In [ ]:
import pandas as pd
import numpy as np

%tensorflow_version 2.x
import tensorflow as tf
from tensorflow.keras import layers

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
from google.colab import drive
#drive.mount('/content/drive')
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
!cp -r /content/drive/MyDrive/Project/Lda2vec-Tensorflow-master .
!cp /content/drive/MyDrive/Project/cleaned.txt .
!cp /content/drive/MyDrive/Project/papers.csv .

In [ ]:
import sys
sys.path.append('/content/Lda2vec-Tensorflow-master')

In [ ]:
from lda2vec import *

Instructions for updating:
non-resource variables are not supported in the long term


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X

## Find Number of Documents with Each Year

In [ ]:
data = pd.read_csv("papers.csv")

In [ ]:
data = data.sort_values(by=["year"], kind='mergesort')

In [ ]:
years = list(data["year"].unique())
counts = list(data["year"].value_counts())[::-1]
for i in range(1,len(counts)):
  counts[i] += counts[i-1]
year_index = {years[i]:counts[i] for i in range(len(years))}

In [ ]:
year_index

{1987: 90,
 1988: 184,
 1989: 285,
 1990: 412,
 1991: 552,
 1992: 695,
 1993: 839,
 1994: 989,
 1995: 1139,
 1996: 1290,
 1997: 1442,
 1998: 1594,
 1999: 1746,
 2000: 1904,
 2001: 2101,
 2002: 2299,
 2003: 2503,
 2004: 2710,
 2005: 2917,
 2006: 3124,
 2007: 3341,
 2008: 3591,
 2009: 3853,
 2010: 4145,
 2011: 4451,
 2012: 4811,
 2013: 5179,
 2014: 5582,
 2015: 5993,
 2016: 6562,
 2017: 7241}

## Preprocessing

In [ ]:
df = pd.read_csv("cleaned.txt")
df = df[df["stop_removed_paper_text"].notnull()]

In [ ]:
df = df[6562:7241] # Use only 2017 papers

In [ ]:
# Initialize a preprocessor
P = nlppipe.Preprocessor(df, "stop_removed_paper_text", max_features=30000, maxlen=10000, min_count=30)

In [ ]:
# Run the preprocessing on your dataframe
P.preprocess()


---------- Tokenizing Texts ----------


676it [01:12,  9.29it/s] 


Removing 39328 low frequency tokens out of 45170 total tokens

---------- Getting Skipgrams ----------


676it [00:59, 11.33it/s]


In [ ]:
# Save data to data_dir
P.save_data("clean_data", embedding_matrix=None)

## Using the LDA2Vec Model

Use separate model if you wish to test on other sets of documents

In [ ]:
# Path to preprocessed data
data_path  = "clean_data"
# Whether or not to load saved embeddings file
load_embeds = True

# Load data from files
(idx_to_word, word_to_idx, freqs, pivot_ids,
 target_ids, doc_ids) = utils.load_preprocessed_data(data_path, load_embed_matrix=False)

In [ ]:
# Number of unique documents
num_docs = doc_ids.max() + 1
# Number of unique words in vocabulary (int)
vocab_size = len(freqs)
# Number of topics to cluster into
num_topics = 20
# Amount of iterations over entire dataset
num_epochs = 100
# Batch size - Increase/decrease depending on memory usage
batch_size = 4096
# Epoch that we want to "switch on" LDA loss
switch_loss_epoch = 0
# If True, save logdir, otherwise don't
save_graph = True

In [ ]:
# Initialize the model
m = model(num_docs,
          vocab_size,
          num_topics,
          embedding_size=128,
          pretrained_embeddings=None,
          freqs=freqs,
          batch_size = batch_size,
          save_graph_def=save_graph)

In [ ]:
# Train the model
m.train(pivot_ids,
        target_ids,
        doc_ids,
        len(pivot_ids),
        num_epochs,
        idx_to_word=idx_to_word,
        switch_loss_epoch=switch_loss_epoch)


EPOCH: 1
LOSS 1941.62 w2v 4.8553553 lda 1936.7646

EPOCH: 2
LOSS 1924.3671 w2v 4.7416286 lda 1919.6255

EPOCH: 3
LOSS 1924.8082 w2v 5.187755 lda 1919.6205

EPOCH: 4
LOSS 1924.6147 w2v 4.994253 lda 1919.6205

EPOCH: 5
LOSS 1927.3643 w2v 7.743763 lda 1919.6205
Instructions for updating:
keep_dims is deprecated, use keepdims instead
---------Closest 10 words to given indexes----------
Topic 0 : particular, similar, specifically, contrast, need, based, approach, example, addition, finally
Topic 1 : particular, similar, specifically, contrast, need, based, approach, example, addition, finally
Topic 2 : particular, similar, specifically, contrast, need, based, approach, addition, example, finally
Topic 3 : particular, similar, specifically, need, contrast, based, approach, example, addition, finally
Topic 4 : particular, similar, specifically, contrast, need, based, approach, example, addition, finally
Topic 5 : particular, similar, specifically, contrast, need, based, approach, example, ad